In [ ]:
#1: import some necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlt
import seaborn as sns

In [ ]:
#2: load data
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train = pd.read_csv("/kaggle/input/aiforcegen3-challenge1/train.csv")
print(train.shape)
test = pd.read_csv("/kaggle/input/aiforcegen3-challenge1/test.csv")
print(test.shape)

In [ ]:
train.head()

In [ ]:
y_train = train["label"]
X_train = train.drop(labels = ["label"], axis = 1)

In [ ]:
plt.figure(figsize = (15, 7))
g = sns.countplot(y_train, palette = "icefire")
plt.title("Number of digit classes")
print(" \t # y_train value counts # \n", y_train.value_counts())

In [ ]:
X_train = X_train / 255.0
X_test = test / 255.0

In [ ]:
X_test = X_test.values.reshape(-1,28,28,1)
X_train = X_train.values.reshape(-1,28,28,1)

In [ ]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
Y_train = to_categorical(y_train, num_classes = 10)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)
print("x_train shape",X_train.shape)
print("x_val shape",X_val.shape)
print("y_train shape",Y_train.shape)
print("y_val shape",Y_val.shape)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'Same', 
                 activation = 'relu', input_shape = (28,28,1)))
#filter: số lớp Conv, padding: tạo vòng 0 quanh model, 28 28: kích thước, 1: 1 chiều sâu
model.add(MaxPool2D(pool_size = (2,2)))
#pool_size: kích thước ma trận pooling
model.add(Dropout(0.25))
#dropout: loại bỏ một phần dữ liệu
model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))
model.add(Dropout(0.25))

# fully connected
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))
model.summary()

In [ ]:
optimizer = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics = ["accuracy"])
epochs = 100  
batch_size = 250

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False, 
        rotation_range=5,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False,  
        vertical_flip=False)  

datagen.fit(X_train)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=6)
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val), steps_per_epoch=X_train.shape[0] // batch_size,
                              callbacks=[early_stopping])

In [ ]:
print("Accuracy of the model is --> " , model.evaluate(X_val, Y_val, batch_size=batch_size)[1]*100 , "%")
print("Loss of the model is --> " , model.evaluate(X_val, Y_val, batch_size=batch_size)[0])

In [ ]:
labels = model.predict(X_test)
labels.shape

In [ ]:
labels = np.argmax(labels, axis=1)
labels.shape

In [ ]:
submission = pd.read_csv("../input/aiforcegen3-challenge1/sample.csv")
submission

In [ ]:
df.to_csv('submission.csv', index = False)
index = test.index.values
data = {'ImageId' : index, "Label" : labels}
df = pd.DataFrame(data = data)
df.head()
submit = pd.DataFrame({'ImageId' : index, "Label" : labels.astype(int).ravel()})
submit.to_csv("submission.csv",index = False)
submit